In [2]:
%load_ext autoreload
%autoreload 2

In [156]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models
import pandas as pd
import sys
sys.path.append('..')
from utils import preprocess, get_windows
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [3]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

In [5]:
nlp = spacy.load('en')

In [183]:
#dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
#docs = dataset['data']
#docs

dataset = pd.read_csv('Documents_Complete.csv')
docs = []

for row_val in dataset['body']:
    val = str(row_val).split(',')
    docs.append(''.join(val))

discoveri complic search vaccin team armi doctor said uncov interact viru viru protect smallpox physician walter reed armi institut research said appar expos viru develop diseas receiv smallpox vaccin militari test recruit vaccin find report england journal medicin signific scientist begun work vaccin reli smallpox vaccin case report rais provoc question concern ultim safeti vaccin said group robert report also throw question belief held scientist smallpox vaccin expos peopl milder protect form diseas known cowpox modifi protect peopl host diseas


In [186]:
# number of documents
len(docs)

20578

In [187]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)



















  0%|          | 0/20578 [00:00<?, ?it/s]















  0%|          | 5/20578 [00:00<08:53, 38.57it/s]















  0%|          | 10/20578 [00:00<08:22, 40.93it/s]















  0%|          | 17/20578 [00:00<07:26, 46.10it/s]















  0%|          | 26/20578 [00:00<06:42, 51.08it/s]















  0%|          | 33/20578 [00:00<06:14, 54.84it/s]















  0%|          | 40/20578 [00:00<05:57, 57.42it/s]















  0%|          | 48/20578 [00:00<05:37, 60.77it/s]















  0%|          | 54/20578 [00:00<06:02, 56.56it/s]















  0%|          | 61/20578 [00:01<06:03, 56.43it/s]















  0%|          | 67/20578 [00:01<05:58, 57.14it/s]















  0%|          | 74/20578 [00:01<05:49, 58.66it/s]















  0%|          | 82/20578 [00:01<05:24, 63.12it/s]















  0%|          | 93/20578 [00:01<04:57, 68.79it/s]















  0%|          | 101/20578 [00:01<04:49, 70.68it/s]















  1

  3%|▎         | 697/20578 [00:08<04:41, 70.66it/s]















  3%|▎         | 705/20578 [00:08<04:31, 73.21it/s]















  3%|▎         | 715/20578 [00:08<04:12, 78.82it/s]















  4%|▎         | 724/20578 [00:08<04:03, 81.61it/s]















  4%|▎         | 733/20578 [00:08<04:13, 78.36it/s]















  4%|▎         | 742/20578 [00:08<04:06, 80.37it/s]















  4%|▎         | 751/20578 [00:09<04:00, 82.60it/s]















  4%|▎         | 763/20578 [00:09<03:38, 90.85it/s]















  4%|▍         | 773/20578 [00:09<04:02, 81.50it/s]















  4%|▍         | 784/20578 [00:09<03:44, 88.02it/s]















  4%|▍         | 794/20578 [00:09<03:38, 90.52it/s]















  4%|▍         | 805/20578 [00:09<03:27, 95.07it/s]















  4%|▍         | 815/20578 [00:09<03:39, 89.85it/s]















  4%|▍         | 825/20578 [00:09<03:38, 90.47it/s]















  4%|▍         | 838/20578 [00:09<03:20, 98.49it/s]












  7%|▋         | 1395/20578 [00:16<04:17, 74.62it/s]















  7%|▋         | 1405/20578 [00:16<03:59, 79.96it/s]















  7%|▋         | 1415/20578 [00:16<03:46, 84.67it/s]















  7%|▋         | 1425/20578 [00:17<03:37, 88.09it/s]















  7%|▋         | 1438/20578 [00:17<03:16, 97.23it/s]















  7%|▋         | 1450/20578 [00:17<03:12, 99.37it/s]















  7%|▋         | 1461/20578 [00:17<03:31, 90.52it/s]















  7%|▋         | 1471/20578 [00:17<03:27, 91.99it/s]















  7%|▋         | 1481/20578 [00:17<03:31, 90.40it/s]















  7%|▋         | 1491/20578 [00:17<03:43, 85.51it/s]















  7%|▋         | 1501/20578 [00:17<03:36, 88.29it/s]















  7%|▋         | 1510/20578 [00:17<03:35, 88.39it/s]















  7%|▋         | 1520/20578 [00:18<03:32, 89.75it/s]















  7%|▋         | 1530/20578 [00:18<03:34, 88.73it/s]















  7%|▋         | 1539/20578 [00:18<03:45, 84.33i

 10%|▉         | 2050/20578 [00:25<03:40, 84.01it/s]















 10%|█         | 2061/20578 [00:25<03:25, 90.25it/s]















 10%|█         | 2072/20578 [00:25<03:15, 94.51it/s]















 10%|█         | 2082/20578 [00:25<03:42, 83.07it/s]















 10%|█         | 2091/20578 [00:25<04:08, 74.36it/s]















 10%|█         | 2102/20578 [00:25<03:54, 78.89it/s]















 10%|█         | 2114/20578 [00:25<03:35, 85.80it/s]















 10%|█         | 2124/20578 [00:26<04:04, 75.59it/s]















 10%|█         | 2135/20578 [00:26<03:42, 82.96it/s]















 10%|█         | 2144/20578 [00:26<03:44, 82.08it/s]















 10%|█         | 2153/20578 [00:26<03:56, 77.76it/s]















 11%|█         | 2163/20578 [00:26<03:48, 80.54it/s]















 11%|█         | 2172/20578 [00:26<04:00, 76.66it/s]















 11%|█         | 2180/20578 [00:26<04:48, 63.72it/s]















 11%|█         | 2187/20578 [00:26<05:09, 59.41i

 13%|█▎        | 2754/20578 [00:33<04:20, 68.47it/s]















 13%|█▎        | 2765/20578 [00:33<03:55, 75.53it/s]















 13%|█▎        | 2774/20578 [00:33<04:39, 63.81it/s]















 14%|█▎        | 2782/20578 [00:34<05:02, 58.90it/s]















 14%|█▎        | 2793/20578 [00:34<04:22, 67.88it/s]















 14%|█▎        | 2801/20578 [00:34<04:17, 69.17it/s]















 14%|█▎        | 2812/20578 [00:34<03:50, 77.05it/s]















 14%|█▎        | 2821/20578 [00:34<03:51, 76.86it/s]















 14%|█▍        | 2830/20578 [00:34<04:17, 68.92it/s]















 14%|█▍        | 2840/20578 [00:34<03:56, 75.12it/s]















 14%|█▍        | 2849/20578 [00:34<03:50, 76.89it/s]















 14%|█▍        | 2861/20578 [00:35<03:29, 84.53it/s]















 14%|█▍        | 2870/20578 [00:35<03:42, 79.61it/s]















 14%|█▍        | 2882/20578 [00:35<03:29, 84.64it/s]















 14%|█▍        | 2891/20578 [00:35<03:40, 80.36i

...

 83%|████████▎ | 17142/20578 [03:02<00:25, 134.22it/s]















 83%|████████▎ | 17160/20578 [03:02<00:23, 143.29it/s]















 83%|████████▎ | 17180/20578 [03:02<00:21, 156.36it/s]















 84%|████████▎ | 17197/20578 [03:02<00:21, 154.56it/s]















 84%|████████▎ | 17214/20578 [03:02<00:21, 153.16it/s]















 84%|████████▎ | 17230/20578 [03:02<00:21, 154.36it/s]















 84%|████████▍ | 17246/20578 [03:02<00:23, 143.76it/s]















 84%|████████▍ | 17261/20578 [03:02<00:23, 141.31it/s]















 84%|████████▍ | 17276/20578 [03:03<00:25, 128.10it/s]















 84%|████████▍ | 17290/20578 [03:03<00:25, 129.77it/s]















 84%|████████▍ | 17305/20578 [03:03<00:25, 130.73it/s]















 84%|████████▍ | 17322/20578 [03:03<00:23, 139.79it/s]















 84%|████████▍ | 17337/20578 [03:03<00:23, 139.80it/s]















 84%|████████▍ | 17352/20578 [03:03<00:23, 138.00it/s]















 84%|████████▍ | 173

 88%|████████▊ | 18126/20578 [03:10<00:35, 68.61it/s]















 88%|████████▊ | 18134/20578 [03:10<00:37, 64.97it/s]















 88%|████████▊ | 18141/20578 [03:10<00:42, 57.19it/s]















 88%|████████▊ | 18153/20578 [03:10<00:36, 67.25it/s]















 88%|████████▊ | 18162/20578 [03:10<00:33, 71.22it/s]















 88%|████████▊ | 18173/20578 [03:11<00:30, 77.91it/s]















 88%|████████▊ | 18183/20578 [03:11<00:29, 80.18it/s]















 88%|████████▊ | 18192/20578 [03:11<00:32, 73.89it/s]















 88%|████████▊ | 18200/20578 [03:11<00:35, 67.46it/s]















 88%|████████▊ | 18208/20578 [03:11<00:35, 66.54it/s]















 89%|████████▊ | 18217/20578 [03:11<00:32, 71.87it/s]















 89%|████████▊ | 18231/20578 [03:11<00:27, 84.04it/s]















 89%|████████▊ | 18242/20578 [03:11<00:26, 89.67it/s]















 89%|████████▊ | 18252/20578 [03:11<00:25, 92.21it/s]















 89%|████████▉ | 18265/20578 [03:1

 92%|█████████▏| 18914/20578 [03:18<00:16, 102.18it/s]















 92%|█████████▏| 18929/20578 [03:18<00:14, 110.84it/s]















 92%|█████████▏| 18941/20578 [03:18<00:15, 106.20it/s]















 92%|█████████▏| 18952/20578 [03:19<00:16, 98.82it/s] 















 92%|█████████▏| 18963/20578 [03:19<00:18, 87.93it/s]















 92%|█████████▏| 18974/20578 [03:19<00:17, 92.95it/s]















 92%|█████████▏| 18984/20578 [03:19<00:17, 92.96it/s]















 92%|█████████▏| 18994/20578 [03:19<00:16, 93.50it/s]















 92%|█████████▏| 19009/20578 [03:19<00:14, 105.26it/s]















 92%|█████████▏| 19021/20578 [03:19<00:14, 107.75it/s]















 92%|█████████▏| 19033/20578 [03:19<00:15, 101.52it/s]















 93%|█████████▎| 19046/20578 [03:19<00:14, 106.79it/s]















 93%|█████████▎| 19058/20578 [03:20<00:15, 97.35it/s] 















 93%|█████████▎| 19069/20578 [03:20<00:15, 98.37it/s]















 93%|█████████▎| 19081/20

 96%|█████████▌| 19730/20578 [03:26<00:11, 74.22it/s]















 96%|█████████▌| 19738/20578 [03:27<00:11, 71.26it/s]















 96%|█████████▌| 19749/20578 [03:27<00:10, 76.88it/s]















 96%|█████████▌| 19758/20578 [03:27<00:11, 74.45it/s]















 96%|█████████▌| 19766/20578 [03:27<00:13, 62.07it/s]















 96%|█████████▌| 19775/20578 [03:27<00:11, 67.65it/s]















 96%|█████████▌| 19784/20578 [03:27<00:11, 69.32it/s]















 96%|█████████▌| 19793/20578 [03:27<00:11, 70.91it/s]















 96%|█████████▌| 19801/20578 [03:27<00:11, 70.61it/s]















 96%|█████████▋| 19813/20578 [03:28<00:09, 80.23it/s]















 96%|█████████▋| 19824/20578 [03:28<00:08, 87.20it/s]















 96%|█████████▋| 19835/20578 [03:28<00:08, 89.95it/s]















 96%|█████████▋| 19846/20578 [03:28<00:07, 94.39it/s]















 96%|█████████▋| 19856/20578 [03:28<00:07, 91.74it/s]















 97%|█████████▋| 19867/20578 [03:2

number of additionally removed short documents: 2727
total number of tokens: 616937
counts most common [('produc', 1750), ('feder', 1644), ('credit', 1620), ('statement', 1619), ('declin', 1599), ('approv', 1597), ('contract', 1586), ('announc', 1572), ('reserv', 1570), ('american', 1552), ('growth', 1551), ('earn', 1548), ('agre', 1545), ('level', 1543), ('term', 1535), ('chairman', 1531), ('program', 1526), ('cost', 1523), ('commiss', 1514), ('payment', 1510), ('currenc', 1488), ('hous', 1482), ('depart', 1476), ('point', 1447), ('chang', 1421), ('compar', 1420), ('recent', 1418), ('franc', 1408), ('polici', 1400), ('reduc', 1387), ('cash', 1372), ('talk', 1366), ('west', 1364), ('japanes', 1360), ('yesterday', 1345), ('estim', 1344), ('budget', 1337), ('low', 1334), ('mark', 1334), ('like', 1310), ('remain', 1299), ('purchas', 1292), ('sharehold', 1285), ('lead', 1275), ('period', 1273), ('agricultur', 1268), ('investor', 1262), ('accord', 1257), ('rais', 1256), ('plant', 1254), ('b

In [189]:
#print("encoded_docs ", len(encoded_docs[6][1])) # Token ids per document
print("decoder ",decoder[0])
print("word_counts ", word_counts)

decoder  produc
word_counts  [1750, 1644, 1620, 1619, 1599, 1597, 1586, 1572, 1570, 1552, 1551, 1548, 1545, 1543, 1535, 1531, 1526, 1523, 1514, 1510, 1488, 1482, 1476, 1447, 1421, 1420, 1418, 1408, 1400, 1387, 1372, 1366, 1364, 1360, 1345, 1344, 1337, 1334, 1334, 1310, 1299, 1292, 1285, 1275, 1273, 1268, 1262, 1257, 1256, 1254, 1253, 1244, 1240, 1237, 1230, 1228, 1227, 1220, 1220, 1218, 1212, 1207, 1204, 1203, 1196, 1195, 1191, 1186, 1183, 1183, 1181, 1177, 1172, 1170, 1154, 1154, 1134, 1133, 1133, 1121, 1105, 1105, 1101, 1101, 1099, 1084, 1080, 1079, 1078, 1077, 1074, 1074, 1073, 1068, 1067, 1061, 1052, 1049, 1045, 1037, 1035, 1032, 1032, 1020, 1014, 1014, 1012, 1012, 1011, 1011, 994, 994, 994, 981, 973, 970, 963, 959, 959, 958, 957, 954, 952, 951, 948, 944, 943, 939, 938, 934, 924, 912, 906, 906, 899, 897, 895, 886, 884, 880, 880, 878, 872, 872, 870, 869, 868, 860, 859, 858, 855, 852, 851, 850, 846, 846, 846, 843, 842, 839, 838, 832, 831, 827, 826, 826, 825, 818, 816, 815, 805, 799, 

In [190]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}
print(doc_decoder)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 9, 9: 10, 10: 11, 11: 12, 12: 15, 13: 16, 14: 17, 15: 18, 16: 19, 17: 21, 18: 25, 19: 26, 20: 27, 21: 29, 22: 30, 23: 32, 24: 33, 25: 34, 26: 35, 27: 36, 28: 37, 29: 38, 30: 39, 31: 40, 32: 41, 33: 42, 34: 43, 35: 44, 36: 45, 37: 46, 38: 47, 39: 48, 40: 49, 41: 50, 42: 51, 43: 53, 44: 54, 45: 55, 46: 56, 47: 57, 48: 58, 49: 59, 50: 60, 51: 62, 52: 63, 53: 65, 54: 66, 55: 67, 56: 68, 57: 69, 58: 70, 59: 71, 60: 73, 61: 74, 62: 75, 63: 77, 64: 79, 65: 80, 66: 81, 67: 82, 68: 84, 69: 85, 70: 87, 71: 92, 72: 95, 73: 96, 74: 97, 75: 98, 76: 100, 77: 102, 78: 103, 79: 105, 80: 106, 81: 107, 82: 108, 83: 109, 84: 110, 85: 111, 86: 112, 87: 114, 88: 116, 89: 118, 90: 119, 91: 120, 92: 121, 93: 123, 94: 124, 95: 125, 96: 126, 97: 127, 98: 128, 99: 129, 100: 130, 101: 131, 102: 132, 103: 133, 104: 134, 105: 135, 106: 136, 107: 137, 108: 138, 109: 139, 110: 142, 111: 144, 112: 145, 113: 146, 114: 150, 115: 151, 116: 152, 117: 153, 118: 155, 119:

In [191]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    #print("--------------------------------")
    #print(windows)
    data += [[index, w[0]] + w[1] for w in windows]
    #print(data)

data = np.array(data, dtype='int64')
#print("final data")
print(data[0])

















0it [00:00, ?it/s]















178it [00:00, 1122.59it/s]















1997it [00:00, 1505.81it/s]















3891it [00:00, 1970.26it/s]















6058it [00:00, 2708.64it/s]















6974it [00:01, 2976.25it/s]















9080it [00:01, 4008.97it/s]















10242it [00:01, 4022.05it/s]















12396it [00:01, 5319.98it/s]















12408it [00:01, 8269.24it/s]

[   0  214  791 1314  238  434  212  656  142 1314 1140 2607]


In [192]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [193]:
# number of windows (equals to the total number of tokens)
data.shape[0]

616937

In [194]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

In [195]:
#%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

In [196]:
# number of unique words
vocab_size

3361

In [197]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
#print(texts[0])
dictionary = corpora.Dictionary(texts)
#print(dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]

print(corpus[2])

[(9, 1), (12, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 3), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 3), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 7), (102, 3), (103, 1), (104, 1)]


In [198]:
decoder[0]

'produc'

In [202]:
#%%time
n_topics = 20
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

In [203]:
print(corpus_lda[100])

[(0, 0.030956334), (1, 0.025111528), (2, 0.027048802), (3, 0.028799824), (4, 0.030377047), (5, 0.030757852), (6, 0.03698121), (7, 0.036093507), (8, 0.029317008), (9, 0.028956713), (10, 0.032609105), (11, 0.06119392), (12, 0.057086945), (13, 0.02711517), (14, 0.123446114), (15, 0.044413563), (16, 0.026895713), (17, 0.029881652), (18, 0.026126627), (19, 0.26683134)]


In [204]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : point facil basi gold credit save mortgag underwrit yield salomon
topic 1 : japanes ministri talk commun european diplomat cabl west telecommun tariff
topic 2 : comput softwar ship line avail introduc network person machin ventur
topic 3 : franc earn store union french divis steel partner acquir purchas
topic 4 : moodi subordin investor asset chase convert standard debentur public rais
topic 5 : sharehold right merger approv tender acquir vote cocoa buffer director
topic 6 : soybean corn commod cent bushel program agricultur depart usda cash
topic 7 : coffe reagan hous uganda white american resign chairman staff statement
topic 8 : barrel crude contract drug test index energi effect trader malaysia
topic 9 : brazil banker payment creditor credit repay commerci negoti philippin line
topic 10 : china interst citi texa earn daili land depart beef subsidiari
topic 11 : baker secretari farm chairman support jame credit help confer senat
topic 12 : deficit budget hous democrat spen